# Live network threat detection

### Import necessary libraries

In [ ]:
import os
import pandas as pd
import joblib
import time
import sys
import logging
from cicflowmeter.sniffer import create_sniffer
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

### Define the directory to watch

In [ ]:
# Define the path to the traffic file
traffic_file_path = '../traffic_data/flows.csv'
# Define the path to the file where the last processed position or timestamp is stored
checkpoint_file_path = '../traffic_data/last_processed_checkpoint.ckpt'
# Define the path to the anomaly history file
anomaly_history_file = '../traffic_data/anomaly_history.csv'

In [ ]:

# Configuration
# INTERFACE = "eth5"
# LOG_FILE = "../traffic_data/cicflowmeter_output.log"

# def start_capturing():
#     # 1. Setup Logging (Equivalent to: -v > ... 2>&1)
#     # We configure the logger to write to the file instead of the console
#     logging.basicConfig(
#         filename=LOG_FILE,
#         level=logging.DEBUG, # Equivalent to -v (verbose)
#         format='%(asctime)s | %(levelname)s | %(message)s',
#         filemode='w' # Overwrite mode (use 'a' to append)
#     )
    
#     # Also redirect standard stdout/stderr to the log file to capture 
#     # any raw print statements or crash errors
#     sys.stdout = open(LOG_FILE, 'a')
#     sys.stderr = sys.stdout

#     print(f"Starting CICFlowMeter on {INTERFACE}...")

#     # 2. Create and Start the Sniffer
#     try:
#         # Equivalent to: cicflowmeter -i wlan0 -c traffic_data/flows.csv
#         sniffer = create_sniffer(
#             input_file=None,    # None means live capture
#             input_interface=INTERFACE,
#             output_mode='flow',     # Ensure we are outputting flow data
#             output=traffic_file_path, 
#             verbose=True            # Helper for internal library logging
#         )
        
#         # This will run indefinitely until stopped (Ctrl+C or kill signal)
#         # sniffer.start()

#     except PermissionError:
#         logging.error("Permission denied. You must run this script with sudo.")
#     except Exception as e:
#         logging.exception(f"An error occurred: {e}")

### Load the trained model

In [ ]:
model = joblib.load('../models/rf_classifier.pkl')  # Change 'your_trained_model.pkl' to the path of your trained model file

### Function to read the last processed position or timestamp from the checkpoint file

In [ ]:
def read_checkpoint():
    """
    Read the last processed position or timestamp from the checkpoint file.

    Returns:
        int: Last processed position or timestamp.
    """
    if os.path.exists(checkpoint_file_path):
        with open(checkpoint_file_path, 'r') as f:
            try:
                return int(f.read())
            except ValueError:
                return 1104
    else:
        return 1104  # Start from the beginning of the file if checkpoint file doesn't exist

### Function to write the last processed position or timestamp to the checkpoint file

In [ ]:
def write_checkpoint(position):
    """
    Write the last processed position or timestamp to the checkpoint file.

    Args:
        position (int): Last processed position or timestamp.
    """
    with open(checkpoint_file_path, 'w') as f:
        f.write(str(position))

### Function to write anomalies to the anomaly history CSV file

In [ ]:
def write_anomalies_to_csv(anomalies):
    """
    Write anomalies to the anomaly history CSV file.

    Args:
        anomalies (list): List of anomaly information dictionaries.
    """
    if not os.path.exists(anomaly_history_file):
        with open(anomaly_history_file, 'w') as f:
            f.write("Timestamp,Anomaly\n")  # Write header if file doesn't exist
    df = pd.DataFrame(anomalies)
    df.to_csv(anomaly_history_file, mode='a', index=False, header=False)  # Append to file without writing header again

### Define function to preprocess data and make predictions

In [ ]:
def predict_anomalies(new_data):
    """
    Predict anomalies in the new data.

    Args:
        new_data (DataFrame): New data to be analyzed.

    Returns:
        list: List of anomaly information dictionaries.
    """
    # Preprocess the new data
    df = new_data.drop(columns=['destination_port', 'protocol', 'timestamp', 'source_ip', 'destination_ip', 'source_port', 'cwr_flag_count']).sort_index(axis=1)

    # Make predictions on the new data
    predictions = model.predict(df)  # Assuming 'label' is the target column and is not included in the features
    
    # Initialize list to store anomalies
    anomalies = []

    for i, prediction in enumerate(predictions):
        if prediction != 0:
            labelEncoder = joblib.load('../models/label_encoder.joblib')
            anomaly_info = {
                "timestamp": new_data.loc[i, 'timestamp'],
                "anomaly": labelEncoder.inverse_transform([prediction])[0]
            }
            anomalies.append(anomaly_info)
            print(f"🔴 Anomaly detected: {anomaly_info['anomaly']} at {anomaly_info['timestamp']}")
    if anomalies == []:
        print("🟢 No anomalies detected")

    return anomalies

### Define the event handler for the file system

In [ ]:
class MyHandler(FileSystemEventHandler):
    """
    Handler class to detect file modifications and trigger anomaly detection.
    """
    def on_modified(self, event):
        """
        Method called when a file is modified.
        """
        if event.src_path == traffic_file_path:
            print("File modified. Detecting anomalies...")
            
            # Read the last processed position from the checkpoint file
            last_processed_position = read_checkpoint()
            
            # Read the new data from the traffic file, starting from the last processed position
            with open(traffic_file_path, 'r') as f:
                header = f.readline().strip('\n').split(',')

                # Move the file pointer to the last processed position
                f.seek(last_processed_position)

                # Read the data from the file, starting from the last processed position
                data = f.readlines()

                # Combine header with data
                combined_data = [row.strip('\n').split(',') for row in data]

                # Create a DataFrame from the combined data
                new_data = pd.DataFrame(combined_data, columns=header)

                # Retrieve the current file position
                current_position = f.tell()
            
            # Trigger prediction function
            try:
                anomalies = predict_anomalies(new_data)
                if anomalies:
                    write_anomalies_to_csv(anomalies)
            except Exception as e:
                pass
            
            # Update the last processed position in the checkpoint file
            write_checkpoint(current_position)

### Set up file system event observer

In [ ]:

event_handler = MyHandler()
observer = Observer()
observer.schedule(event_handler, path=traffic_file_path, recursive=False)
observer.start()

In [ ]:
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()